#### Mahri


In [ ]:
# On g collab only
!git clone https://github.com/electronjia/heart_cardiac_mri_image_processing.git

In [ ]:
# On g collab only
!pip install pydicom

In [ ]:
# On g collab only
!pwd
!ls -l
%cd heart_cardiac_mri_image_processing/edge_detection_and_contours
!pwd

In [ ]:
import pandas as pd
from config import *
import os
import numpy as np
from skimage import exposure, filters, measure, morphology
import matplotlib.pyplot as plt
import pydicom
from skimage import img_as_ubyte
from skimage.feature import canny
from skimage.measure import label, regionprops
import ipywidgets as widgets
from IPython.display import display
from skimage.draw import polygon


import cv2
import json
from skimage.morphology import disk
from skimage.filters import rank
from skimage import measure
from scipy.spatial import distance
from skimage.morphology import convex_hull_image

In [ ]:
# On g collab only
import sys
sys.path.append('/content/heart_cardiac_mri_image_processing/edge_detection_and_contours')
user_handle = r"/content/heart_cardiac_mri_image_processing/data"
patient_data_excel_path = r"/content/heart_cardiac_mri_image_processing/scd_patientdata_xlsx.xlsx"
patient_data = "patient_data"
patient_filepaths = "patient_filepaths"

user_handle_g_collab = "/content/heart_cardiac_mri_image_processing/data"

In [ ]:
# Number of columns for plotting
num_cols = 10

# Patient index
patient_idx = 0

patient_thresholds = {
    'eccentricity': [0.8, 0.8, 0.8, 0.8, 0.8],
    'area': [300, 400, 400, 400, 400],
    'seed_starter' : [(120, 90), (130,80), (120,90), (120,90), (120,90)],
    'disk_size': [1,1,1,1,1],
    'sigma_gaus': [0.2,0.2,0.2,0.2,0.2],
    'clip_limit': [0.03,0.03,0.03,0.03,0.03],
    'sigma_edge': [7.00,6.00,6.00,5.00,6.00],
    'level_val': [0.7,0.7,0.7,0.7,0.7],
    'mask_info_dict': []

}


# Each patient will have their own mask info dict



In [ ]:
patient_xlsx = patient_data_excel_path
patient_data_df = pd.read_excel(patient_xlsx, sheet_name=patient_data)
patient_filepaths_df = pd.read_excel(patient_xlsx, sheet_name=patient_filepaths)
patient_mask_filepaths = pd.read_excel(patient_xlsx, sheet_name="mask_filepaths")

display(patient_data_df.head(2))
display(patient_filepaths_df.head(2))

In [ ]:
# Single patient

patient_ids = patient_filepaths_df['patient_id'].unique()
patient_id = patient_ids[patient_idx]
single_patient_filepaths = patient_filepaths_df.loc[patient_filepaths_df['patient_id'] == patient_id, 'dcm_image_filepath'].tolist()


In [ ]:
def read_convert_dicom_img(filepath):
    # Read the DICOM image
    dicom_img = pydicom.dcmread(filepath)
    
    # Extract the pixel data and convert to 8-bit image
    img = img_as_ubyte(dicom_img.pixel_array / np.max(dicom_img.pixel_array))
    return img

In [ ]:
def apply_contrast_enhancement(img, clip_limit):
    # Apply contrast
    img_contrast = exposure.equalize_adapthist(img, clip_limit=clip_limit)

    # Create radial mask for center contrast enhancement
    # Create a grid of distances from the center
    rows, cols = img_contrast.shape
    center_row, center_col = rows // 2, cols // 2

    # Create radial distance map
    y, x = np.ogrid[:rows, :cols]
    distance_from_center = np.sqrt((x - center_col)**2 + (y - center_row)**2)

    # Normalize distance to range [0, 1]
    distance_from_center = distance_from_center / np.max(distance_from_center)

    # Apply the mask (higher values in the center)
    radial_mask = 1 - distance_from_center  # Invert to get higher values at the center
    enhanced_contrast_img = img_contrast * radial_mask

    return enhanced_contrast_img

In [ ]:
def detect_edges_and_contours(img, sigma, level):
    # Edge detection
    img_edges = canny(img, sigma=sigma)

    # Find and draw contours
    contours = measure.find_contours(img_edges, level=level)

    # Create a binary mask with contours
    binary_mask = np.zeros_like(img, dtype=np.uint8)
    for contour in contours:
        # Convert contour to polygon coordinates and fill it in the binary image
        contour_points = contour.astype(int)
        rr, cc = polygon(contour_points[:, 0], contour_points[:, 1], shape=binary_mask.shape)
        binary_mask[rr, cc] = 255  # Set contour area to foreground (255)

    return img_edges, contours, binary_mask

In [ ]:
def get_labels_regions(binary_mask):
    # Find the labels in binary mask
    labeled_img, _ = measure.label(binary_mask, connectivity=2, return_num=True)

    # Measure region properties
    regions = measure.regionprops(labeled_img)

    return labeled_img, regions

In [ ]:
def segment_left_ventricle(filepath, labeled_img, regions, mask_info_dict, img_idx, seed_starter, eccentricity_th, area_th):

  # Find best region depending on previous best region or 
  min_distance = float('inf')
  best_region = None

  # Attempt to get the best region by comparing to previous region if exists, if not, use seed starter
  for prop in regions:
      # Compare the centroid from the dictionary if it exists
      if mask_info_dict['centroid_coords']:
          previous_centroid = np.array(mask_info_dict['centroid_coords'][img_idx-1])
          current_centroid = np.array(prop.centroid)
          distance = np.linalg.norm(current_centroid - previous_centroid)  # Euclidean distance

          if distance < min_distance:  
              min_distance = distance
              best_region = prop
      else:
        # Use the set seed starter
        previous_centroid = np.array(seed_starter)
        current_centroid = np.array(prop.centroid)
        distance = np.linalg.norm(current_centroid - previous_centroid)  # Euclidean distance

        if distance < min_distance:  
            min_distance = distance
            best_region = prop


  # Attempt to evaluate the best region's eccentricity and area according to set thresholds
  if best_region.eccentricity < eccentricity_th and best_region.area < area_th:
    filled_mask = get_convex_hull_mask(labeled_img, best_region)

  else:
    best_region = None
    min_distance = float('inf')
    for prop in regions:
        
      previous_centroid = np.array(seed_starter)
      current_centroid = np.array(prop.centroid)
      distance = np.linalg.norm(current_centroid - previous_centroid)  # Euclidean distance

      if distance < min_distance:  
          min_distance = distance
          best_region = prop

    if best_region.eccentricity > eccentricity_th or best_region.area > area_th:
      filled_mask = 0

    else:
      filled_mask = get_convex_hull_mask(labeled_img, best_region)

     
  # Append the dictionary info
  mask_info_dict["index"].append(img_idx)
  mask_info_dict["frames"].append(filepath)
  mask_info_dict["eccentricity"].append(best_region.eccentricity)
  mask_info_dict["area"].append(best_region.area)
  mask_info_dict["coords"].append(best_region.coords)
  mask_info_dict["centroid_coords"].append(best_region.centroid)
  mask_info_dict["distance_centroid"].append(min_distance)
  mask_info_dict['mask'].append(filled_mask)

  return filled_mask, mask_info_dict

In [7]:
def get_convex_hull_mask(labeled_img, region):
        
    # Create a mask for smallest eccentricity region
    binary_mask = np.zeros_like(labeled_img, dtype=np.uint8)
    binary_mask[labeled_img == region.label] = 255

    # Generate convex hull mask
    hull_mask = convex_hull_image(binary_mask)

    # Find contours in the binary mask
    contours = measure.find_contours(hull_mask, level=0.5)

    # Obtain the filled mask
    filled_mask = np.zeros_like(binary_mask, dtype=np.uint8)

    # Loop through each contour and fill it in the mask
    for contour in contours:
        # Convert contour coordinates to integer values
        contour_points = contour.astype(int)

        # Get the coordinates of the contour and fill the polygon
        rr, cc = polygon(contour_points[:, 0], contour_points[:, 1], shape=filled_mask.shape)
        
        # Set the region inside the polygon to 255 (foreground)
        filled_mask[rr, cc] = 255  # Set filled region to white (255)

    return filled_mask

In [ ]:
# The code for a single patient

print(f"Processing patient: {patient_id}")

img_processing_params = {key: value[patient_idx] for key,value in patient_thresholds.items()}

mask_info_dict = {
    "index": [],
    "frames": [],
    "eccentricity": [],
    "area": [],
    "coords": [],
    "centroid_coords": [],
    "distance_centroid": [],
    "mask": []
}
for batch_idx, batch_start in enumerate(range(0, len(single_patient_filepaths), num_cols)):

    batch_filepaths = single_patient_filepaths[batch_start:batch_start + num_cols]

    num_imgs = len(batch_filepaths)
    fig, axes = plt.subplots(1, num_imgs, figsize=(num_imgs * 2, 4))

    if num_imgs == 1:
        axes = [axes]  # Ensure axes is iterable when there's only one image
    
    # Iterate over the filepaths in given batch
    for batch_img_idx, img_filepath in enumerate(batch_filepaths):

        # Get the original image index
        img_idx = int(f"{batch_idx}{batch_img_idx}")
        
        # For Google Colab only
        img_filepath = img_filepath.replace("\\", "/")
        img_abs_filepath = os.path.join(user_handle_g_collab, img_filepath)

        # Get the 8 bit image
        img_8bit = read_convert_dicom_img(img_abs_filepath)

        # Get filtered image
        img_filt = filters.gaussian(img_8bit, sigma=img_processing_params['sigma_gaus'])

        # Apply radial contrast on image
        img_contrast = apply_contrast_enhancement(img_filt, clip_limit=img_processing_params['clip_limit'])

        # Get edges, contours, and binary mask
        img_edges, contours, binary_mask = detect_edges_and_contours(img_contrast, sigma=img_processing_params['sigma_edge'], level=img_processing_params['level'])

        # Get labeled image and regions
        labeled_img, regions = get_labels_regions(binary_mask)

        # Get the filled mask where left ventricle is defined
        filled_mask, mask_info_dict = segment_left_ventricle(filepath=img_abs_filepath, labeled_img=labeled_img, regions=regions, mask_info_dict=mask_info_dict, img_idx=img_idx, seed_starter=img_processing_params['seed_starter'], eccentricity_th = img_processing_params['eccentricity'], area_th=img_processing_params['area'])

patient_thresholds['mask_info_dict'].append(mask_info_dict)


        